In [2]:
!pip install zeep requests lxml suds-py3 icecream


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


### Importing Libraries
This cell imports the necessary libraries for interacting with the SOAP web service, parsing XML, and debugging.
- `zeep`: A modern SOAP client for Python.
- `requests`: Used by `zeep` for making HTTP requests.
- `xml.etree.ElementTree`: For parsing XML responses.
- `suds-py3`: An alternative SOAP client for Python 3.
- `icecream`: For easy debugging print statements.

In [3]:
from zeep import Client
from zeep.transports import Transport
from requests import Session
from requests.auth import HTTPBasicAuth
import xml.etree.ElementTree as ET
from suds.client import Client as SudsClient
from icecream import ic
import os
import logging

# Configure logging for

In [5]:
def create_soap_client(wsdl_url):
    """Creates a Zeep SOAP client with custom settings."""
    session = Session()
    # Optional: Add basic authentication if required
    # session.auth = HTTPBasicAuth('user', 'password')

    transport = Transport(session=session)
    client = Client(wsdl_url, transport=transport)
    return client


### Find Station Code (Zeep)
This function finds the station

In [7]:
def find_station_code(client, station_name):
    """Finds the station code for a given station name."""
    try:
        result = client.service.exportStationCodesXML()
        root = ET.fromstring(result)
        for station in root.findall('.//station'):
            name = station.find('stationname').text
            code = station.find('stationcode').text
            if name == station_name:
                return code
        return None
    except Exception as e:
        ic(f"Error finding station code: {e}")
        return None


### Export All Parameters (Zeep)
This function exports all parameters for a given station code by calling the `exportAllParamsXML

In [8]:
def export_all_params(client, station_code):
    """Exports all parameters for a given station code."""
    try:
        result = client.service.exportAllParamsXML(station_code)
        return result
    except Exception as e:
        ic(f"Error exporting parameters: {e}")
        return None


SyntaxError: expected ':' (2839230251.py, line 6)

### Main Execution (Zeep)
This section defines the WSDL URL, creates the Zeep SOAP client, finds the station code for "Tijuana", exports its parameters, and prints the results. Error

In [10]:
# Define the WSDL URL
wsdl_url = 'https://cdmo.baruch.sc.edu/webservices2/requests.cfc?wsdl'
station_to_find = "Tijuana"

# Create the SOAP client
zeep_client = create_soap_client(wsdl_url)

if zeep_client:
    ic(f"Zeep client created successfully for WSDL: {wsdl_url}")

    # Find the station code for "Tijuana"
    tijuana_code = find_station_code(zeep_client, station_to_find)

    if tijuana_code:
        ic(f"Station code for {station_to_find}: {tijuana_code}")

        # Export all parameters for the station
        station_params_xml = export_all_params(zeep_client, tijuana_code)

        if station_params_xml:
            ic(f"Parameters for {station_to_find} (XML):")
            print(station_params_xml)
        else:
            ic(f"Could not retrieve parameters for {station_to_find}.")
    else:
        ic(f"Could not find station code for {station_to_find}.")


/Users/valentin/development/dev_earthcube/infrastructuredashboard/.venv/lib/python3.12/site-packages/zeep/wsdl/wsdl.py:346: UserWarning: The wsdl:message for '{http://webservices2}exportSingleParamXMLResponse' contains an invalid part ('exportSingleParamXMLReturn'): invalid xsd type or elements
  warnings.warn(str(exc))
/Users/valentin/development/dev_earthcube/infrastructuredashboard/.venv/lib/python3.12/site-packages/zeep/wsdl/wsdl.py:346: UserWarning: The wsdl:message for '{http://webservices2}exportAllParamsDateRangeXMLNewResponse' contains an invalid part ('exportAllParamsDateRangeXMLNewReturn'): invalid xsd type or elements
  warnings.warn(str(exc))
/Users/valentin/development/dev_earthcube/infrastructuredashboard/.venv/lib/python3.12/site-packages/zeep/wsdl/wsdl.py:346: UserWarning: The wsdl:message for '{http://webservices2}exportAllParamsDateRangeXMLResponse' contains an invalid part ('exportAllParamsDateRangeXMLReturn'): invalid xsd type or elements
  warnings.warn(str(exc))


### Suds-py3 Client Demonstration
This section demonstrates the use of the `suds-py3` client. It initializes the

Let me help you handle the missing namespace issue in SOAP requests. I'll add code to demonstrate namespace handling and error mitigation.



### Handling Missing XML-SOAP Namespace
When dealing with SOAP services that require the "http://xml.apache.org/xml-soap" namespace, we can use custom namespace bindings and transport settings to handle this issue.

In [ ]:
from zeep.plugins import HistoryPlugin
from zeep.xsd.const import XML_NS
from zeep.exceptions import Fault
from zeep.wsse.username import UsernameToken

# Create a history plugin to track SOAP messages
history = HistoryPlugin()


# Custom transport with namespace handling
def create_soap_client_with_namespaces(wsdl_url):
    session = Session()
    transport = Transport(session=session)

    # Define additional namespaces
    settings = {
        'xsd': 'http://www.w3.org/2001/XMLSchema',
        'soap': 'http://schemas.xmlsoap.org/soap/envelope/',
        'xml-soap': 'http://xml.apache.org/xml-soap'
    }

    # Create client with custom settings
    client = Client(
        wsdl_url,
        transport=transport,
        plugins=[history],
        strict=False,
        xml_huge_tree=True
    )

    # Add namespaces to the client's context
    client.transport.session.headers['Content-Type'] = 'text/xml;charset=UTF-8'

    return client, history

In [ ]:
# Test the client with namespace handling
try:
    enhanced_client, history = create_soap_client_with_namespaces(wsdl_url)

    # Make a test call
    result = enhanced_client.service.exportStationCodesXML()

    # Print the last sent envelope for debugging
    last_sent = history.last_sent
    if last_sent:
        print("Last sent SOAP envelope:")
        print(last_sent['envelope'])

except Fault as e:
    ic(f"SOAP Fault: {e}")
except Exception as e:
    ic(f"Error: {e}")

### Error Handling Tips
If you still encounter namespace issues:
1. The code above adds explicit namespace bindings
2. Sets strict mode to False for more lenient XML parsing
3. Includes debugging with HistoryPlugin to inspect SOAP messages
4. Uses enhanced transport settings with proper content type headers


In [23]:
# Using suds-py3 client with xml-soap namespace
try:
    # Create Suds client with custom namespace
    from suds.xsd.doctor import Import, ImportDoctor
    try:
        # Define the missing namespace and its location (if you have a local copy or alternative source)
        missing_namespace = "http://xml.apache.org/xml-soap"
        # If you have a local copy of the schema, provide the path:
        # location = "file:///path/to/local/apache-soap.xsd"
        # If there's no direct replacement, you might just need to tell SUDS to ignore or handle it loosely:
        location = None # Or a dummy location if needed

        # Create an Import object
        imp = Import(missing_namespace, location)
        doctor = ImportDoctor(imp)
    except Exception as e:
        print('Error creating Import object:', e)


    # Create an ImportDoctor and register the import


    suds_client = SudsClient(wsdl_url, doctor=doctor)
    suds_client.options.transport.options.timeout = 300



    ic("Suds-py3 client created successfully.")

    ic("Available Suds-py3 methods:")
    print(suds_client)

    # Call a sample method using suds-py3
    suds_result = suds_client.service.exportStationCodesXML()
    ic("Result from suds-py3 exportStationCodesXML")
    ic(suds_result)
except Exception as e:
    ic(f"Error using Suds-py3 client: {e}")


ic| 'Suds-py3 client created successfully.'
ic| 'Available Suds-py3 methods:'



Suds ( https://github.com/cackharot/suds-py3 )  version: 1.4.5.0 IN  build: 20211115

Service ( RequestsService ) tns="http://webservices2"
   Prefixes (2)
      ns0 = "http://rpc.xml.coldfusion"
      ns1 = "http://schemas.xmlsoap.org/soap/encoding/"
   Ports (1):
      (requests.cfc)
         Methods (11):
            NERRFilterStationCodesXML(xs:string NERRFilter, )
            NERRFilterStationCodesXMLNew(xs:string NERRFilter, )
            exportAllParamsDateRangeXML(xs:string Station_Code, xs:string minDate, xs:string maxDate, xs:string param, )
            exportAllParamsDateRangeXMLNew(xs:string Station_Code, xs:string minDate, xs:string maxDate, xs:string param, xs:string wildcard, )
            exportAllParamsXML(xs:string Station_Code, xs:string recs, )
            exportAllParamsXMLNew(xs:string Station_Code, xs:string recs, xs:string wildcard, )
            exportSingleParamXML(xs:string Station_Code, xs:string recs, xs:string param, )
            exportSingleParamXMLNew(

ic| f"Error using Suds-py3 client: {e}": ("Error using Suds-py3 client: Type not found: '(Document, "
                                          "http://xml.apache.org/xml-soap, )'")


### Get Service Information (Suds-py3)
This section gets and prints service information using the `suds-py

In [14]:
try:
    ic("Suds-py3 service information:")
    print(suds_client.sd[0].ports[0].methods)
except Exception as e:
    ic(f"Error getting Suds-py3 service information: {e}")

ic| 'Suds-py3 service information:'
ic| f"Error getting Suds-py3 service information: {e}": ("Error getting Suds-py3 service information: 'tuple' object has no attribute "
                                                         "'methods'")


In [25]:
from suds.client import Client

soapClient = Client("http://cdmo.baruch.sc.edu/webservices2/requests.cfc?wsdl", timeout=90, retxml=True)

#Get the station codes SOAP request example.
station_codes = soapClient.service.exportStationCodesXML()
print (station_codes)

#Get all parameters from the station NIWOLMET for the date range of 2014-12-30 to 2014-12-31
params = soapClient.service.exportAllParamsDateRangeXML('niwolmet', '2014-12-30', '2014-12-31', '*')
print (params)



b'<?xml version="1.0" encoding="utf-8"?><soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n <soapenv:Body>\n  <ns1:exportStationCodesXMLResponse soapenv:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/" xmlns:ns1="http://webservices2">\n   <exportStationCodesXMLReturn xsi:type="ns2:Document" xmlns:ns2="http://xml.apache.org/xml-soap">\n    <returnData>\n    \t\t    <data>Invalid ip 76.175.204.235</data>\n\n    \t</returnData>\n   </exportStationCodesXMLReturn>\n  </ns1:exportStationCodesXMLResponse>\n </soapenv:Body>\n</soapenv:Envelope>'
b'<?xml version="1.0" encoding="utf-8"?><soapenv:Envelope xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n <soapenv:Body>\n  <ns1:exportAllParamsDateRangeXMLResponse soapenv:encodingStyle="http://schemas.xmlsoa